### Feature engineering phase 1

- Remove unnecessary columns to reduce dataframe size
- Filter train records according to min/max langtitude and longitude in grid info
- Generate grid info and aggregate records accroding to VIN and grid id

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import timeit
%matplotlib inline

# loading training files
ecar_training_data_path = 'data/training/ecar/'
rcar_training_data_path = 'data/training/rcar/'

ecar_output_filepath = 'data/training/ecar/processed/ecar_training.csv'
rcar_output_filepath = 'data/training/rcar/processed/rcar_training.csv'

weather_filepath = 'data/weather.csv'
training_features_filepath = 'data/training.csv'
test_features_filepath = 'data/test.csv'
submission_filepath = 'data/submit_samples.csv'

ecar_filenames = [f for f in os.listdir(ecar_training_data_path)]
rcar_filenames = [f for f in os.listdir(rcar_training_data_path)]

# define drop columns
ecar_drop_columns = ['work_mode','mileage','avg_fuel_consumption','system_mode']
rcar_drop_columns = ['power_mode','mileage','fuel_consumption']
common_drop_columns = ['lat','lon','date_time']

# training features
feature_columns = ['year','month','day','hour','weekday','grid_id',
                   'temperture','rainy','holiday','visibility','wind','car_number']

# loading grid min/max lantitude and longitude
grids = pd.read_csv('data/grid_info.csv')
grid_max_lantitude = grids['latitude_to'].max()
grid_min_lantitude = grids['latitude_from'].min()
grid_max_longitude = grids['longitude_to'].max()
grid_min_longitude = grids['longitude_from'].min()

In [2]:
def round_down_lat_lon(df):
    df['lat'] = df.lat.round(3)
    df['lon'] = df.lon.round(3)
    return df
        
def filter_data_by_lan_lon(df):
    df = df[(df.lat >= grid_min_lantitude ) & (df.lat <= grid_max_lantitude)]
    df = df[(df.lon >= grid_min_longitude ) & (df.lon <= grid_max_longitude)]
    return df

def add_grid_id(df):
    df_with_grid_id = pd.DataFrame(index=[0], columns=['car_id', 'date_time','speed','lat','lon','grid_id'])
    df_with_grid_id = df_with_grid_id.fillna(-1)
    for i in range(len(grids)):
        min_lan, max_lan = grids['latitude_from'][i], grids['latitude_to'][i]
        min_lon, max_lon, grid_id = grids['longitude_from'][i], grids['longitude_to'][i], grids['grid_id'][i]
        df_ = df[(df.lat >= min_lan ) & (df.lat <= max_lan)]
        df_ = df_[(df_.lon >= min_lon ) & (df_.lon <= max_lon)]
        df_['grid_id'] = grid_id
        df_with_grid_id = df_with_grid_id.append(df_)
    return df_with_grid_id

def format_date(date_time):
    return date_time[:-9].replace('-','')

def retrieve_hour(date_time):
    return date_time[-8:-6]

def preprocess_data(dataType='rcar'):
    # init run time
    start = timeit.default_timer()
    # define common variables
    filenames = rcar_filenames
    filepath = rcar_training_data_path
    drop_columns = rcar_drop_columns
    output_filepath = rcar_output_filepath    
    
    if dataType == 'ecar':
        filenames = ecar_filenames
        filepath = ecar_training_data_path
        drop_columns = ecar_drop_columns
        output_filepath = ecar_output_filepath
        
    # if file is already exist return
    if os.path.exists(output_filepath):
        print(output_filepath + " already existed, will return directly")
        return
    
    df_ = pd.DataFrame(index=[0], columns=['car_id', 'date','hour','speed', 'grid_id'])
    df_ = df_.fillna(-1)
    for filename in filenames:
        # if is directory skip
        if os.path.isdir(filepath + filename):
            continue
        # loading data
        df = pd.read_csv(filepath + filename, low_memory=False)
        # drop uncessary columns
        df = df.drop(columns=drop_columns)
        # filter data by lat and lon
        df = filter_data_by_lan_lon(df)
        # format data_time column
        df['hour'] = df['date_time'].apply(lambda x : retrieve_hour(x))
        df['date'] = df['date_time'].apply(lambda x : format_date(x))
        # generate new column grid_id
        df = add_grid_id(df)
        # filter data by grid_id
        df = df[df.grid_id != -1]
        # drop date_time, lat and lon
        df = df.drop(columns=common_drop_columns)
        # group by below column and get average speed for each record
        df = df.groupby(['car_id', 'date','hour','grid_id'], as_index=False).mean()
        # round average speed
        df.speed = df.speed.round(3)
        # appending df
        df_ = df_.append(df)
        # remove grid_id = -1
        df_ = df_[df_.grid_id != -1]
    # print log and total time
    print("Finish process all the file, total cost:[", timeit.default_timer() - start ,"] seconds.")
    # write final file
    df_.to_csv(output_filepath)

In [15]:
preprocess_data('rcar')

Finish all the file, total cost:[ 344.8129758159416 ]


In [27]:
preprocess_data('ecar')

data/training/ecar/processed/ecar_training.csv already existed, will return directly


### Feature engineering phase 2
- Merge ecar and rcar training data
- Drop column speed and car_id first(these 2 columns maybe used in future but let's try a simple solution first)
- Aggregate training data and calculate total number of cars per day/per hour/per grid
- Merge training data with weather data

In [11]:
def preprocess_weather_data():
    # split date to year, month and day, adding one more column weekday
    df = pd.read_csv(weather_filepath)
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    # format date yyyy-mm-dd to yyyymmdd for later join with car's data
    df['date'] = df['date'].astype(str)
    df['date'] = df['date'].apply(lambda x : x.replace('-', ''))
    df['date'] = df['date'].astype(np.int64)
    # drop uncessary columns
    #df = df.drop(columns=['visibility', 'wind'])
    return df

def preprocess_phase1_data():
    # loading files
    ecar = pd.read_csv(ecar_output_filepath)
    rcar = pd.read_csv(rcar_output_filepath)
    weather = preprocess_weather_data()
    full_data = generate_full_data_set()
    # merge weather with full data
    f_weather = pd.merge(full_data, weather, on=['year','month','day','hour'], how='left')    
    # merge ecar and rcar data
    df = ecar.append(rcar)
    # remove unnamed column
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    # drop car id and speed
    df = df.drop(columns=['car_id','speed'])
    # add column car_number
    df['car_number'] = 1
    # aggregate data by date,hour and grid id
    df = df.groupby(['date','hour','grid_id'], as_index=False).sum()
    # remove unecessary row
    df = df[(df.date != 20161230 ) & (df.date != 20170101)]
    # merge data set
    df = pd.merge(f_weather, df, on=['date','hour','grid_id'], how='left')
    #df = pd.merge(df, weather, on=['date','hour'], how='left')
    # fill NA car number to 0
    df.fillna(0, inplace = True)
    # write to disk
    df.to_csv(training_features_filepath, columns=feature_columns, index=False)

# train data from 2017-01-02 to 2017-03-12
def generate_full_data_set():
    jan_df = generate_data_by_range(2017,1,2,31,0,23,1,50)
    feb_df = generate_data_by_range(2017,2,1,28,0,23,1,50)
    march_df = generate_data_by_range(2017,3,1,12,0,23,1,50)
    df = jan_df.append(feb_df)
    df = df.append(march_df)
    return df
    
def generate_data_by_range(year, month, start_date, end_date, start_time, end_time, start_gid, end_gid):
    # define arrays
    years = []
    months = []
    days = []
    hours = []
    grid_id = []
    # generate grid id, date and hour according to criteria
    for date in range(start_date, end_date+1):
        for time in range(start_time, end_time+1):
            for gid in range(start_gid, end_gid+1):
                years.append(year)
                months.append(month)
                days.append(date)
                hours.append(time)
                grid_id.append(gid)
    # genreate dataframe            
    df = pd.DataFrame(data={'year':year,'month':month,'day':days, 'hour': hours, 'grid_id':grid_id})
    df['flag'] = 0
    df=df.reindex(columns=['year','month','day','hour','grid_id','flag'])
    return df

In [12]:
preprocess_phase1_data()
features = pd.read_csv(training_features_filepath)
features.head()

,year,month,day,hour,weekday,grid_id,temperture,rainy,holiday,visibility,wind,car_number
0,2017,1,2,0,0,1,11,0,1,1,0,0.0
1,2017,1,2,0,0,2,11,0,1,1,0,1.0
2,2017,1,2,0,0,3,11,0,1,1,0,0.0
3,2017,1,2,0,0,4,11,0,1,1,0,5.0
4,2017,1,2,0,0,5,11,0,1,1,0,3.0


### Generating test features for submission

- from 20170313 to 20170326 2 weeks data
- 9<=hour<=22's data

In [9]:
def generate_sample_submission_file(start_date, end_date, start_time, end_time, start_gid, end_gid):
    # define arrays
    grid_id = []
    dates = []
    hours = []
    # generate grid id, date and hour according to criteria
    for date in range(start_date, end_date+1):
        for time in range(start_time, end_time+1):
            for gid in range(start_gid, end_gid+1):
                grid_id.append(gid)
                dates.append('201703' + str(date))
                hours.append(time)
    # genreate dataframe            
    df = pd.DataFrame(data={'grid_id': grid_id, 'date': dates, 'hour': hours})
    df['car_number'] = 0
    df=df.reindex(columns=['grid_id','date','hour','car_number'])
    # write to csv
    df.to_csv(submission_filepath, index=False) 


def preprocess_test_data():
    test = pd.read_csv(submission_filepath)
    weather = preprocess_weather_data()
    df = pd.merge(test, weather, on=['date', 'hour'], how='left')
    df.to_csv(test_features_filepath, columns=feature_columns, index=False)

In [10]:
generate_sample_submission_file(13, 26, 9, 22, 1, 50)

preprocess_test_data()

test = pd.read_csv(test_features_filepath)

test.head()

,year,month,day,hour,weekday,grid_id,temperture,rainy,holiday,visibility,wind,car_number
0,2017,3,13,9,0,1,9,1,0,1,1,0
1,2017,3,13,9,0,2,9,1,0,1,1,0
2,2017,3,13,9,0,3,9,1,0,1,1,0
3,2017,3,13,9,0,4,9,1,0,1,1,0
4,2017,3,13,9,0,5,9,1,0,1,1,0


In [29]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 12 columns):
year          9800 non-null int64
month         9800 non-null int64
day           9800 non-null int64
hour          9800 non-null int64
weekday       9800 non-null int64
grid_id       9800 non-null int64
temperture    9800 non-null int64
visibility    9800 non-null int64
wind          9800 non-null int64
rainy         9800 non-null int64
holiday       9800 non-null int64
car_number    9800 non-null int64
dtypes: int64(12)
memory usage: 918.8 KB
